# Søk om utsatt frist for et set av klienter

In [ ]:
from docs.test.testinnsending.skatteetaten_api import get_access_token
from requests import post
from faker import Faker

token = get_access_token()

In [ ]:
def lag_indentifikator(identifikator: str) -> dict():
    fake = Faker()

    if len(identifikator) > 9:
        identifikator_dict = {"foedselsnummer" : identifikator}
    else:
        identifikator_dict = {"organisasjonsnummer" : identifikator}

    kandidat = {
        "identifikator": identifikator_dict,
        "naering":  True,
        "navn": fake.name(),
        "epost": fake.email()
    }
    return kandidat

identifikatorer = ["27867196108", "310783145"]
klienter = [lag_indentifikator(kandidat) for kandidat in identifikatorer]

In [ ]:
inntektsår = "2024"
identifikator = "27867196108"
url_path = f"/api/skattemelding/v2/utsattfristsoeknad/{identifikator}"
#url = "http://ekstern-token-validering.fastsetting.formueinntekt.skattemelding.sit.apps.utv10.paas.skead.no"
base_url = "https://idporten-api-sbstest.sits.no"

In [ ]:
utsatt_frist_request = post(base_url+url_path, headers=token, json=klienter)

In [ ]:
print(utsatt_frist_request.status_code, utsatt_frist_request.text)